# MGL 869 - Hive - Groupe 8 - Lab 

---


## Packages

In [1]:
import pandas as pd
import os
import git
import re
from concurrent.futures import ThreadPoolExecutor, as_completed, ProcessPoolExecutor
import subprocess

## Global variables

In [2]:
HIVE_CSV_path = 'ApacheHiveCSV'
HIVE_CSV_separator = '^'
GIT_HIVE_path = 'repo'
GIT_url = 'git@github.com:apache/hive.git'

## OS environment

In [3]:
os.makedirs('output', exist_ok=True)
current_project_path = os.getcwd()

## Git environment

In [4]:
# Current project path
current_project_path = os.getcwd()
# Parent directory
parent_directory = os.path.dirname(current_project_path)
# Repository path
repo_path = current_project_path + '/' + GIT_HIVE_path
# Repository url
rep_url = GIT_url

## Extract data from Hive

### Function to create dataframe from multiple sources

In [5]:
def createHiveDataFrame():
    csv_files = [os.path.join(HIVE_CSV_path, file) for file in os.listdir(HIVE_CSV_path) if file.endswith('.csv')]
    dataframes = [pd.read_csv(file, sep="^") for file in csv_files]
    
    dataframes_filtered = []
    for df in dataframes:
        fix_version_columns = [col for col in df.columns if col.startswith('Fix Version/s')]
        affects_version_columns = [col for col in df.columns if col.startswith('Affects Version/s')]
    
        df['Fix Versions Combined'] = df[fix_version_columns].apply(lambda x: ', '.join(x.dropna().astype(str)), axis=1)
        df['Affects Versions Combined'] = df[affects_version_columns].apply(lambda x: ', '.join(x.dropna().astype(str)),
                                                                            axis=1)
    
        # Supprimer les colonnes originales
        df = df.drop(fix_version_columns, axis=1)
        df = df.drop(affects_version_columns, axis=1)
    
        keep: list = ['Issue key', 'Status', 'Resolution', 'Created', 'Fix Versions Combined', 'Affects Versions Combined']
        df = df.loc[:, keep]
    
        dataframes_filtered.append(df)
    return dataframes_filtered

### Create dataframe

In [6]:
df_merged = pd.concat(createHiveDataFrame(), ignore_index=True, sort=False)

### Save dataframe in a file

In [7]:
df_merged.to_csv('output/filtered_data.csv')

### Collect bug ids

In [8]:
# Create a set of all the ids
ids = set(df_merged[df_merged.columns[df_merged.columns.str.contains('Issue key')]].values.flatten())

## Git research
### Clone if repo isn't already clone

In [9]:
# Check if repo exists
if not os.path.exists(repo_path) or not os.path.exists(os.path.join(repo_path, '.git')):
    repo = git.Repo.clone_from(repo_url, repo_path)
else:
    repo = git.Repo(repo_path)

### Regex to find bug names in commits

In [10]:
pattern = re.compile(r'HIVE-(\d{3,5})')

### Function to find files modified for a list of commit

In [11]:
# Function to process a batch of commits
def process_commits(commits):
    print(f'Processing {len(commits)} commits')
    local_repo = git.Repo(repo_path)
    couples = []
    for key in commits:
        commit_id = key
        for match in commits[key]:
            hive_key = f'HIVE-{match}'
            if hive_key in ids:
                for file in local_repo.commit(commit_id).stats.files:
                    couples.append((hive_key, file))
                
    print(f'Finished processing {len(commits)} commits')
    return couples

### Prepare research in batch bug research in commits

In [12]:
num_threads = os.cpu_count() # Get the number of threads of the CPU
chunk_size = len(list(repo.iter_commits())) // num_threads

# Get all commits and files
all_commits = [{} for _ in range(num_threads)]

for i, commit in enumerate(repo.iter_commits()):
    matches = pattern.findall(commit.message)
    if matches:
        all_commits[i // chunk_size][commit.hexsha] = matches


In [13]:
# Initialize the list to store all couples
all_couples = []
with ThreadPoolExecutor(max_workers=num_threads) as executor:
    futures = [executor.submit(process_commits, chunk) for chunk in all_commits]
    for future in as_completed(futures):
        couples = future.result()
        all_couples.extend(couples)

Processing 1078 commits
Processing 1054 commits
Processing 1050 commits
Processing 1083 commits
Processing 1084 commits
Processing 1083 commits
Processing 1078 commits
Processing 1083 commits
Processing 1070 commits
Processing 1065 commits
Processing 1005 commits
Processing 1054 commits
Processing 1046 commits
Processing 1017 commits
Processing 1058 commits
Processing 995 commits
Finished processing 1078 commits
Finished processing 1054 commits
Finished processing 1005 commits
Finished processing 1054 commits
Finished processing 1046 commits
Finished processing 1017 commits
Finished processing 995 commits
Finished processing 1058 commits
Finished processing 1084 commits
Finished processing 1050 commits
Finished processing 1083 commits
Finished processing 1083 commits
Finished processing 1078 commits
Finished processing 1083 commits
Finished processing 1065 commits
Finished processing 1070 commits


In [14]:
# Convert the list of tuples into a DataFrame
df_couples = pd.DataFrame(all_couples, columns=['Issue Key', 'File'])
# Save the DataFrame to a CSV file
df_couples.to_csv('couples.csv', index=False)

In [15]:
# Charger le fichier CSV
df_couples = pd.read_csv("couples.csv")
# Garder seulement les fichiers Java ou C++
df_filtered = df_couples[df_couples['File'].str.endswith(('.java', '.cpp', '.c', '.h'))]
# Sauvegarder le fichier filtré si besoin
df_filtered.to_csv("filtered_couples.csv", index=False)

In [16]:
# Obtenir tous les tags et leurs dates de création
tags = repo.tags
versions = {}

for tag in tags:
    # Récupérer la date de création du commit associé au tag
    commit = tag.commit
    commit_date = commit.committed_datetime.strftime('%Y-%m-%d')
    versions[tag.name] = commit_date

# Afficher les versions et leurs dates
print(versions)

{'branch-3.1.2-rc0': '2020-01-13', 'branch-3.1.3-rc0': '2020-01-13', 'master_2015_11_30': '2015-11-29', 'rel/release-1.2.2': '2017-04-01', 'rel/release-2.1.0': '2016-06-17', 'rel/release-2.1.1': '2016-11-29', 'rel/release-2.2.0': '2017-07-21', 'rel/release-2.3.0': '2017-07-13', 'rel/release-2.3.1': '2017-10-19', 'rel/release-2.3.10': '2024-05-04', 'rel/release-2.3.2': '2017-11-09', 'rel/release-2.3.3': '2018-03-28', 'rel/release-2.3.4': '2018-10-31', 'rel/release-2.3.5': '2019-05-07', 'rel/release-2.3.6': '2019-08-13', 'rel/release-2.3.7': '2020-04-07', 'rel/release-2.3.8': '2021-01-06', 'rel/release-2.3.9': '2021-06-01', 'rel/release-3.0.0': '2018-05-17', 'rel/release-3.1.0': '2018-07-23', 'rel/release-3.1.1': '2018-10-23', 'rel/release-3.1.2': '2019-08-22', 'rel/release-3.1.3': '2022-03-24', 'rel/release-4.0.0': '2024-03-21', 'rel/release-4.0.0-alpha-1': '2022-03-22', 'rel/release-4.0.0-alpha-2': '2022-11-07', 'rel/release-4.0.0-beta-1': '2023-08-07', 'rel/release-4.0.1': '2024-09-26

In [17]:
# Filtrer pour garder uniquement les versions principales
filtered_versions = {tag: date for tag, date in versions.items() if tag.startswith('release-') and '-rc' not in tag}
cleaned_versions = {tag.replace('release-', ''): date for tag, date in filtered_versions.items()}
# Afficher les versions principales et leurs dates de sortie
print(cleaned_versions)

{'0.10.0': '2013-01-10', '0.11.0': '2013-05-16', '0.12.0': '2013-10-14', '0.13.0': '2014-04-19', '0.13.1': '2014-06-05', '0.14.0': '2014-11-12', '0.3.0': '2010-10-26', '0.4.0': '2010-10-26', '0.4.1': '2010-10-26', '0.5.0': '2010-10-26', '0.6.0': '2010-10-26', '0.7.0': '2011-03-25', '0.7.1': '2011-06-17', '0.8.0': '2011-12-16', '0.8.1': '2012-01-30', '0.9.0': '2012-04-27', '1.0.0': '2015-02-03', '1.0.1': '2015-05-14', '1.1.0': '2015-03-09', '1.1.1': '2015-05-14', '1.2.0': '2015-05-14', '1.2.1': '2015-06-19', '2.0.0': '2016-02-09', '2.0.1': '2016-05-03'}


In [18]:
# Convertir le dictionnaire des versions en DataFrame
df_versions = pd.DataFrame(list(cleaned_versions.items()), columns=['Version', 'Release Date'])

In [19]:
# Sauvegarder dans un fichier CSV
df_versions.to_csv("versions.csv", index=False)

In [20]:
commits_for_versions = {}
for index, row in df_versions.iterrows():
    version = row['Version']
    date = row['Release Date']
    
    # Obtenir le dernier commit avant la date de version
    commit = repo.git.log('--before', date, '-n', '1', '--pretty=format:%H')
    commits_for_versions[version] = commit

print("Derniers commits pour chaque version :", commits_for_versions)

Derniers commits pour chaque version : {'0.10.0': 'a786579fa0bb3245adea9c19d0da5fbbe7930f64', '0.11.0': '2c87e89b0305f42337070104a0d179c8910696f8', '0.12.0': '811a8b763c35f5c197a94ec311424a077ba2a72b', '0.13.0': 'aaa25fcc41c54767e22bab655dc0096f98fb50f0', '0.13.1': '3fcc32002d8ce3f010cac0423f3cce69a90a4b09', '0.14.0': 'efc8b08672216ca61eee1dfa43187decb094bc81', '0.3.0': 'd8701ea32ab24e4c19d583ba9b40f4789703245b', '0.4.0': 'd8701ea32ab24e4c19d583ba9b40f4789703245b', '0.4.1': 'd8701ea32ab24e4c19d583ba9b40f4789703245b', '0.5.0': 'd8701ea32ab24e4c19d583ba9b40f4789703245b', '0.6.0': 'd8701ea32ab24e4c19d583ba9b40f4789703245b', '0.7.0': 'eb72421c75c56c2fa62527c326f7835e128caca8', '0.7.1': '4f5b4e22cc19e85de8f3bfda29551954bd69a8ee', '0.8.0': '9f2a9ee7b32ed4527d6b1cc2238df535998da2b0', '0.8.1': 'd31122f3f6d77f79e42649e524887bfdce72706f', '0.9.0': '09e9a233748c6b25b5485e6a8b3886bc7bede89d', '1.0.0': 'd4684e994465225380dd604eb0da1a2bff6fb42b', '1.0.1': '31383348c1bd32b7991a20b793a362cca19987e9', 